# Klassifikation und Clustering
Nachdem Texte in einem Vektorraummodell repräsentiert wurden, lassen sie sich mit Hilfe bereits bekannter Methoden klassifizieren oder in Cluster einteilen.

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.cluster import KMeans

## Inhaltsverzeichnis
- [Datensatz](#Datensatz)
- [Beispiel: Klassifikation](#Beispiel-Klassifikation)
- [Beispiel: Clustering](#Beispiel-Clustering)

## Datensatz
Als Beispiel verwenden wir einen fertigen Datensatz, der aus einer Reihe von Nachrichten innerhalb verschiedener, thematisch getrennter Newsgroups bestehen.

In [ ]:
categories = ['sci.space', 'rec.sport.baseball', 'comp.graphics', 'sci.med']
# categories = ['comp.graphics', 'sci.med']
train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

`data` enthält dabei die Nachrichten, während `target` und `target_names` die Zuordnung aufzeigt.

In [ ]:
print(train.data[440].strip())
print('-' * 60)
print(train.target[440], '->', train.target_names[train.target[440]])

Mit der Klasse `TfidfTransformer` lässt sich eine Repräsentation im Vektorraummodell effizient berechnen. (Gegebenenfalls sollte noch eine erweiterte Vorverarbeitung stattfinden.)

In [ ]:
count = CountVectorizer()
tfidf = TfidfTransformer()

X_train_counts = count.fit_transform(train.data)
X_train_tfidf = tfidf.fit_transform(X_train_counts)

X_train_tfidf[440].shape[1]

Die entstehenden Vektoren haben eine Größe von mehr als $36.000$ Elementen.

## Beispiel: Klassifikation
Die Vektoren lassen sich jetzt wie aus vorangegangenen Kapiteln bekannt klassifizieren. Die folgende Zelle trainiert beispielsweise einen einfachen Entscheidungsbaum mit Standardparametern und stellt ihn zur Veranschaulichung der Tiefe einmal grafisch dar.

In [ ]:
tree = DecisionTreeClassifier()
tree.fit(X_train_tfidf, train.target)

plot_tree(tree)
pass

Um unseren trainierten Klassifikator zu testen, erstellen wir neue Dokumente und transformieren diese ebenfalls mit Hilfe des TF-IDF Modells.

In [ ]:
new_documents = [
    'Covid-19 is a bad disease.',
    'OpenGL on the GPU is fast.'
]

X_test_counts = count.transform(new_documents)
X_test_tfidf = tfidf.transform(X_test_counts)

Auch wenn die neuen Dokumente nicht der oben gezeigten Form entsprechen, hoffen wir auf Abstraktion durch die Feature-Extraktion und das Training des Klassifikators.

In [ ]:
prediction = tree.predict(X_test_tfidf)
[train.target_names[p] for p in prediction]

Beachten Sie, dass zu starke Abweichungen vom usprünglichen Trainingsmaterial dennoch zu Fehlklassifikationen führt. Ein Wechsel der Sprache kann beispielsweise schnell zu Fehlklassifikationen führen.

In [ ]:
new_documents_ger = [
    'Covid-19 ist eine schlimme Krankheit.',
    'OpenGL ist schnell auf Grafikkarten.'
]

X_test_ger_counts = count.transform(new_documents_ger)
X_test_ger_tfidf = tfidf.transform(X_test_ger_counts)

prediction = tree.predict(X_test_ger_tfidf)
[train.target_names[p] for p in prediction]

## Beispiel: Clustering
Die Vektoren lassen sich ebenfalls mit den bekannten Methoden zu Clustern zusammenfassen.

In [ ]:
true_k = len(train.target_names)
KMeans(n_clusters=true_k).fit_predict(X_train_tfidf)

In diesem Fall ist $k$ bekannt. Es lassen sich aber natürlich alle Methoden zur Parameterbestimmung oder ein Wechsel zu einem dichtebasierten Verfahren anwenden.